In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option("display.max_columns", None)

2025-04-28 16:30:25.258 | INFO     | inventario_finanzas_int.config:<module>:11 - PROJ_ROOT path is: C:\Users\jarojas\Desktop\Javier\INT\inventario-finanzas-int


In [3]:
df_mobiliario_consolidado = pd.read_excel(
    "../data/raw/MOBILIARIO (ARIEL)/MATRIZ CORRELATIVA AÑO 2025.xlsx"
)

In [4]:
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")

# Lee equipos medicos y los ordena
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")
df_equipos_medicos = df_equipos_medicos.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# lee Industriales, y elimina las columnas completamente vacias
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")
df_industriales = df_industriales.dropna(axis=1, how="all")
df_industriales = df_industriales.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# Lee Informaticos
df_informaticos = pd.read_csv("../data/processed/df_procesada_informaticos.csv")

In [5]:
union_bienes_con_informatico = pd.concat(
    [df_mobiliario, df_equipos_medicos, df_industriales, df_informaticos]
)
union_bienes_con_informatico["conteo"] = 1

union_bienes_sin_informatico = pd.concat([df_mobiliario, df_equipos_medicos, df_industriales])
union_bienes_sin_informatico["conteo"] = 1

In [6]:
conteo_equipos_con_informatico = pd.pivot_table(
    union_bienes_con_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

conteo_equipos_sin_informatico = pd.pivot_table(
    union_bienes_sin_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

## Asignacion de Correlativos

Se hara de forma separada

In [7]:
GLOSAS_EQUIPOS_MEDICOS_CON_CORRELATIVO = ["INT", "INT (SSMO)", "INT (MINSAL)", "COMODATO"]
GLOSAS_INDUSTRIALES_CON_CORRELATIVO = ["INT", "INT (MEL)", "INT (DONACION)", "COMODATO"]

In [8]:
ULTIMO_CORRELATIVO = 4015 + 1
# Identifica los registros a los que hay que asignar correlativo
mask_correlativo_medicos = df_equipos_medicos["propiedad"].isin(
    GLOSAS_EQUIPOS_MEDICOS_CON_CORRELATIVO
)
df_equipos_medicos_con_correlativos = df_equipos_medicos[mask_correlativo_medicos]

# Asigna correlativos a los registros que requieren correlativo
correlativos_equipos_medicos = [
    f"2025-{i}"
    for i in range(
        ULTIMO_CORRELATIVO, ULTIMO_CORRELATIVO + len(df_equipos_medicos_con_correlativos)
    )
]
df_equipos_medicos.loc[mask_correlativo_medicos, "correlativo_2025"] = correlativos_equipos_medicos
df_equipos_medicos["correlativo_2025"] = df_equipos_medicos["correlativo_2025"].replace(
    "nan", np.nan
)

In [9]:
# Aisla el ultimo correlativo asignado
ULTIMO_CORRELATIVO_MEDICOS = int(
    df_equipos_medicos["correlativo_2025"].dropna().iloc[-1].split("-")[-1]
)

PRIMER_CORRELATIVO_INDUSTRIALES = ULTIMO_CORRELATIVO_MEDICOS + 1
# Identifica los registros a los que hay que asignar correlativo
mask_correlativos_ind = df_industriales["propiedad"].isin(GLOSAS_INDUSTRIALES_CON_CORRELATIVO)
df_industriales_con_correlativos = df_industriales[mask_correlativos_ind]

# Asigna los correlativos a los registros que requieren correlativo
correlativos_industriales = [
    f"2025-{i}"
    for i in range(
        PRIMER_CORRELATIVO_INDUSTRIALES,
        PRIMER_CORRELATIVO_INDUSTRIALES + len(df_industriales_con_correlativos),
    )
]

df_industriales.loc[mask_correlativos_ind, "correlativo_2025"] = correlativos_industriales
df_industriales["correlativo_2025"] = df_industriales["correlativo_2025"].replace("nan", np.nan)

In [10]:
# Genera la matriz total de bienes
matriz_total_correlativos = pd.concat([df_equipos_medicos, df_industriales])
matriz_total_correlativos = matriz_total_correlativos.drop(columns="unidad_hoja")

# Reordena columnas de matriz total
orden_columnas = ["correlativo_antiguo", "correlativo_2025"] + list(
    matriz_total_correlativos.columns.drop(["correlativo_antiguo", "correlativo_2025"])
)
matriz_total_correlativos = matriz_total_correlativos[orden_columnas]

In [11]:
# Exporta asignando correlativos
df_equipos_medicos.to_excel("../data/interim/matriz_equipos_medicos_2025.xlsx", index=False)
df_industriales.to_excel("../data/interim/matriz_industriales_2025.xlsx", index=False)

In [12]:
fa.guardar_dataframe_como_tabla_excel(
    matriz_total_correlativos,
    "../data/interim/matriz_bienes_INT_2025.xlsx",
    nombre_tabla="MatrizBienes",
    estilo_tabla="TableStyleMedium1"
)

✅ Archivo guardado exitosamente en: ../data/interim/matriz_bienes_INT_2025.xlsx
